<a href="https://colab.research.google.com/github/iriyagupta/GENAI-BA-CPlus/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

**Recurrent Neural Network Example**
* Data: a series of 1000 numbers
* Learn: the next number in the series

**Recurrent Neural Networks**
* Data in an RNN is fed sequentially
* Each successive element in the sequence is called a "timestep"
* For example, if our data is textual and we have 1000 sentences in the data:
** the sample size is 1000
** the number of features is 1
** the timestep size is the length of the longest sentence
* Data for an RNN must contain both features as well as timesteps

**Example: Arithmetic sequence**
* An arithmetic sequence with a difference of 2 is 1, 3, 5, 7, ......
* Input data to the RNN will consist of subsequence of size n. For n = 4:
** 1, 3, 5, 7 => 9
** 3, 5, 7, 9 => 11
** 5, 7, 9, 11 => 13
** Each sequence has 4 numbers, each number is fed separately, therefore:
** timesteps = 4
** features = 1

## **Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.model_selection import train_test_split

**Generate the data**
* We'll generate an arithmetic sequence of 100000 numbers
* difference between numbers = 3


In [3]:
start = 1
diff = 3
n = 100000
progression = [start + diff*i for i in range (n)]
progression[999]
df = pd.DataFrame(progression)

**Generate RNN input data**
* We'll generate sequences of size k (k=8)
* X = input data
* y = output value (the sume of the input values)
* Note that the data is organized as 1 feature and 8 timesteps

In [4]:
def makeSequence(df, k):
  X, y =[], []
  for i in range(len(df)- k):
    d=i+k
    X.append(df.iloc[i:d,])
    y.append(df.iloc[d,])
  return np.array(X),np.array(y)

X,y = makeSequence(df,8)

In [5]:
X.shape

(99992, 8, 1)

In [ ]:
X[:2]

array([[[ 1],
        [ 4],
        [ 7],
        [10],
        [13],
        [16],
        [19],
        [22]],

       [[ 4],
        [ 7],
        [10],
        [13],
        [16],
        [19],
        [22],
        [25]]])

In [ ]:
y[:2]

array([[25],
       [28]])

**Input shape**
* RNNs require that the input shape be 3 dimensional
* (samples, timesteps, features)
* samples: the number of samples in the training data (length of X_train)
* timesteps: The number of lookback periods


In [6]:
#Understanding timesteps

#Assume our data is:
x_demo = np.array([[1,2],[1,3],[2,3],[5,6],[3,4]])
y_demo = np.array([3,4,5,11,7])

#I.e., y(0) = x(0,0) + x(0,1)

#But, we want y(t) to depend on x(t) and x(t-1)
#We want to use two timesteps to determine y

look_back = 2 #This step plus the previous step

#Since we're looking at 2 timesteps, we're going to have one fewer data items
num_samples = x_demo.shape[0]-look_back + 1
num_features = 2 #The number of features at each timestep

#Create empty arrays for x and y reshaped
x_demo_reshaped = np.zeros((num_samples, look_back, num_features))
y_demo_reshaped = np.zeros((num_samples))
print(y_demo_reshaped)

#Iterate through the data creating x(t-1) and x(t) data for each y
for i in range(num_samples):
    print(i)
    y_position = i + look_back
    x_demo_reshaped[i] = x_demo[i:y_position]
    y_demo_reshaped[i] = y_demo[y_position-1]

x_demo_reshaped,y_demo_reshaped,x_demo_reshaped.shape

[0. 0. 0. 0.]
0
1
2
3


(array([[[1., 2.],
         [1., 3.]],
 
        [[1., 3.],
         [2., 3.]],
 
        [[2., 3.],
         [5., 6.]],
 
        [[5., 6.],
         [3., 4.]]]),
 array([ 4.,  5., 11.,  7.]),
 (4, 2, 2))

**Train and test samples**
* split the data into 70% training and 30% testing

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Reshape input data to (samples, timesteps, features)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1] ))
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))


In [9]:

X_train.shape

(69994, 8, 1)

In [10]:
X_train[:2]

array([[[118483],
        [118486],
        [118489],
        [118492],
        [118495],
        [118498],
        [118501],
        [118504]],

       [[ 90391],
        [ 90394],
        [ 90397],
        [ 90400],
        [ 90403],
        [ 90406],
        [ 90409],
        [ 90412]]])

In [11]:
y_train[:2]

array([[118507],
       [ 90415]])

In [12]:
# SimpleRNN model
model = Sequential()

#units = number of outputs from each RNN
#Note that this refers to the hidden layer outputs not the final y
model.add(SimpleRNN(units=32, input_shape=(8,1), activation="relu"))
model.add(Dense(8, activation="relu"))

#This last layer is the actual y output
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 32)                1088      
                                                                 
 dense (Dense)               (None, 8)                 264       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1361 (5.32 KB)
Trainable params: 1361 (5.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Fit the training data to the model**
* and get the predictions (training and testing)
* and the mean squared error

In [ ]:
model.fit(X_train,y_train, epochs=1000, batch_size=32, verbose=0)
trainPredict = model.predict(X_train)
testPredict= model.predict(X_test)
predicted=np.concatenate((trainPredict,testPredict),axis=0)


938/938 [==============================] - 2s 2ms/step


**

In [ ]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
testScore = model.evaluate(X_test,y_test,verbose=2)
print(trainScore)
print(testScore)

938/938 - 2s - loss: 271061.3438 - 2s/epoch - 2ms/step
274335.09375
271061.34375


In [ ]:
model.predict(X_test)[:3]

938/938 [==============================] - 2s 2ms/step


array([[102228.25 ],
       [ 68791.82 ],
       [ 58546.055]], dtype=float32)

In [ ]:
X_test[:3]

array([[[101896],
        [101899],
        [101902],
        [101905],
        [101908],
        [101911],
        [101914],
        [101917]],

       [[ 68560],
        [ 68563],
        [ 68566],
        [ 68569],
        [ 68572],
        [ 68575],
        [ 68578],
        [ 68581]],

       [[ 58345],
        [ 58348],
        [ 58351],
        [ 58354],
        [ 58357],
        [ 58360],
        [ 58363],
        [ 58366]]])

**Our model doesn't do too well!**
* RNNs require a lot of training
* Let's run it for a larger number of epochs and see if it does better

In [ ]:
model.fit(X_train,y_train, epochs=10000, batch_size=32, verbose=0)
trainPredict = model.predict(X_train)
testPredict= model.predict(X_test)
predicted=np.concatenate((trainPredict,testPredict),axis=0)
trainScore = model.evaluate(X_train, y_train, verbose=0)
testScore = model.evaluate(X_test,y_test,verbose=2)
print(trainScore)
print(testScore)
print(model.predict(X_test)[:3])
print(X_test[:3])